# TAKE VIDEO FROM DISK AND CONVERT TO 3 FPS

In [4]:
# import os
# import ffmpeg   
# HOME = "Video"
# V_PATH = "Videos_L06"
# OUT_DIR = "Video_3fps"
# for V_PATH in os.listdir(HOME):
#     if not os.path.exists(os.path.join(OUT_DIR,V_PATH)):
#         os.makedirs(os.path.join(OUT_DIR,V_PATH))
#     for path in os.listdir(os.path.join(HOME,V_PATH,"video")):
#         print (os.path.join(HOME,V_PATH,"video",path))
#         (
#             ffmpeg
#             .input(os.path.join(HOME,V_PATH,"video",path))
#             .filter('fps', fps=3, round='up')
#             .output(os.path.join(OUT_DIR,V_PATH,path))
#             .run()
#         )

In [5]:
# import os
# os.listdir(os.path.join(HOME,V_PATH))

In [6]:
# import cv2
# from cap_from_youtube import cap_from_youtube

# youtube_url = 'https://youtube.com/watch?v=HNsRpkryGXA'
# cap = cap_from_youtube(youtube_url, 'best')

# cv2.namedWindow('video', cv2.WINDOW_NORMAL)
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     cv2.imshow('video', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


# Extract every 3 frames

In [1]:
import cv2  # still used to save images out
import os
import numpy as np
from decord import VideoReader
from decord import cpu, gpu


def extract_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extract frames from a video using decord's VideoReader
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    assert os.path.exists(video_path)  # assert the video file exists

    # load the VideoReader
    vr = VideoReader(video_path, ctx=cpu(0))  # can set to cpu or gpu .. ctx=gpu(0)
                     
    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = len(vr)
    frames_list = list(range(start, end, every))
    if len(os.listdir(os.path.join(frames_dir, video_filename.split(".")[0]))) ==  len(frames_list):
        return 0
    
    saved_count = 0
    print(f"Total frame(s) count: {end}")

    print(f"Extracting frame(s) count: {len(frames_list)}")

    if every > 25 and len(frames_list) < 1000:  # this is faster for every > 25 frames and can fit in memory
        frames = vr.get_batch(frames_list).asnumpy()

        for index, frame in zip(frames_list, frames):  # lets loop through the frames until the end
            save_path = os.path.join(frames_dir, video_filename.split('.')[0], f"{video_filename}_{frame:05d}.jpg")

            # save_path = os.path.join(frames_dir, video_filename.split(".")[0], "{:010d}.jpg".format(index))  # create the save path
            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                cv2.imwrite(save_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # save the extracted image
                saved_count += 1  # increment our counter by one

    else:  # this is faster for every <25 and consumes small memory
        for index in range(start, end):  # lets loop through the frames until the end
            frame = vr[index]  # read an image from the capture
            if index % every == 0:  # if this is a frame we want to write out based on the 'every' argument
                save_path = os.path.join(frames_dir, video_filename.split('.')[0], f"{video_filename.split('.')[0]}_{index:05d}.jpg")

                # save_path = os.path.join(frames_dir, video_filename.split(".")[0], "{:010d}.jpg".format(index))  # create the save path
                if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                    cv2.imwrite(save_path, cv2.cvtColor(frame.asnumpy(), cv2.COLOR_RGB2BGR))  # save the extracted image
                    saved_count += 1  # increment our counter by one

    return saved_count  # and return the count of the images we saved


def video_to_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extracts the frames from a video
    :param video_path: path to the video
    :param frames_dir: directory to save the frames
    :param overwrite: overwrite frames if they exist?
    :param every: extract every this many frames
    :return: path to the directory where the frames were saved, or None if fails
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    # make directory to save frames, its a sub dir in the frames_dir with the video name
    os.makedirs(os.path.join(frames_dir, video_filename.split(".")[0]), exist_ok=True)
    
    print("Extracting frames from {}".format(video_filename))
    
    extract_frames(video_path, frames_dir, start=start, end=end, every=every)  # let's now extract the frames

    return os.path.join(frames_dir, video_filename)  # when done return the directory containing the frames

In [2]:
# import os

In [3]:
vid_list = os.listdir(r"Videos")
for vid in vid_list:
    for v in os.listdir(os.path.join(r"Videos",vid,"video")):
        video_to_frames(video_path=os.path.join(r"Videos",vid,"video",v), frames_dir='frames_full', overwrite=False, every=3)

Extracting frames from L13_V001.mp4
Extracting frames from L13_V002.mp4
Extracting frames from L13_V003.mp4
Extracting frames from L13_V004.mp4
Extracting frames from L13_V005.mp4
Extracting frames from L13_V006.mp4
Extracting frames from L13_V007.mp4
Extracting frames from L13_V008.mp4
Extracting frames from L13_V009.mp4
Extracting frames from L13_V010.mp4
Extracting frames from L13_V011.mp4
Extracting frames from L13_V012.mp4
Extracting frames from L13_V013.mp4
Extracting frames from L13_V014.mp4
Extracting frames from L13_V015.mp4
Extracting frames from L13_V016.mp4
Extracting frames from L13_V017.mp4
Extracting frames from L13_V018.mp4
Extracting frames from L13_V019.mp4
Extracting frames from L13_V020.mp4
Extracting frames from L13_V021.mp4
Extracting frames from L13_V022.mp4
Extracting frames from L13_V023.mp4
Extracting frames from L13_V024.mp4
Extracting frames from L13_V025.mp4
Extracting frames from L13_V026.mp4
Extracting frames from L13_V027.mp4
Extracting frames from L13_V

In [10]:
!pip install -U sentence-transformers

^C
Traceback (most recent call last):
  File "/home/nghiadangineer/miniconda3/envs/aic/bin/pip", line 11, in <module>
    sys.exit(main())
  File "/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<

In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import torch
#Load CLIP model
# model = SentenceTransformer('clip-ViT-B-32')

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
#Encode an image:
print(device)
img_model = SentenceTransformer('clip-ViT-B-32').to(device)

/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


cuda


/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import os
path_to_keyframe_dir = "pank_keyframes/keyframes"

os.listdir(path_to_keyframe_dir)

['L01_V001',
 'L01_V002',
 'L01_V003',
 'L01_V004',
 'L01_V005',
 'L01_V006',
 'L01_V007',
 'L01_V008',
 'L01_V009',
 'L01_V010',
 'L01_V011',
 'L01_V012',
 'L01_V013',
 'L01_V014',
 'L01_V015',
 'L01_V016',
 'L01_V017',
 'L01_V018',
 'L01_V019',
 'L01_V020',
 'L01_V021',
 'L01_V022',
 'L01_V023',
 'L01_V024',
 'L01_V025',
 'L01_V026',
 'L01_V027',
 'L01_V028',
 'L01_V029',
 'L01_V030',
 'L01_V031',
 'L02_V001',
 'L02_V002',
 'L02_V003',
 'L02_V004',
 'L02_V005',
 'L02_V006',
 'L02_V007',
 'L02_V008',
 'L02_V009',
 'L02_V010',
 'L02_V011',
 'L02_V012',
 'L02_V013',
 'L02_V014',
 'L02_V015',
 'L02_V016',
 'L02_V017',
 'L02_V018',
 'L02_V019',
 'L02_V020',
 'L02_V021',
 'L02_V022',
 'L02_V023',
 'L02_V024',
 'L02_V025',
 'L02_V026',
 'L02_V027',
 'L02_V028',
 'L02_V029',
 'L02_V030',
 'L02_V031',
 'L03_V001',
 'L03_V002',
 'L03_V003',
 'L03_V004',
 'L03_V005',
 'L03_V006',
 'L03_V007',
 'L03_V008',
 'L03_V009',
 'L03_V010',
 'L03_V011',
 'L03_V012',
 'L03_V013',
 'L03_V014',
 'L03_V015',

In [ ]:
import open_clip
open_clip.list_pretrained()

/home/nghiadangineer/miniconda3/envs/aic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [3]:
from PIL import Image
import numpy as np
import os
from tqdm import tqdm, trange
imgs = []
# Nhìn ô code ở trên để hình dung cái folder
# path_to_keyframe_dir = "Keyframes"

if not os.path.exists("clip-vit-B32-features-org"):
    os.mkdir("clip-vit-L14-features-org")

for path in os.listdir(path_to_keyframe_dir):
    imgs = []
    print(path)
    if os.path.exists(os.path.join("clip-vit-B32-features-org",f"{path}.npy")):
        continue
    tmp = []
    dir = os.listdir(os.path.join(path_to_keyframe_dir,path))
    batch_size = 32
    for i in trange(0,len(dir),batch_size):
        batch = dir[i:i+batch_size]
        imgs = []
        for v in batch:
            imgs.append(Image.open(os.path.join(path_to_keyframe_dir,path,v)))
    # for chunk in trange(0,len(imgs),16):
        vid_embed = img_model.encode(imgs)
        if len(tmp):
            tmp = np.concatenate([tmp,vid_embed],axis=0)
        else:
            tmp = vid_embed
    name = path.split(".")[0]
    np.save(os.path.join("clip-vit-B32-features-org",f"{name}.npy"),tmp)
    # break

L01_V001


100%|███████████████████████████████████████████████████████████████████████████████████| 11/11 [00:38<00:00,  3.49s/it]


L01_V002


100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:19<00:00,  2.48s/it]


L01_V003


...

100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:29<00:00,  2.28s/it]


L12_V029


100%|███████████████████████████████████████████████████████████████████████████████████| 11/11 [00:25<00:00,  2.29s/it]


L12_V030


100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:27<00:00,  2.26s/it]


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0
